In [ ]:
# !pip install -qU langchain-graph-retriever
# !pip install "langchain-graph-retriever[chroma]"
# !pip install -qU langchain-community pypdf
# !pip install scikit-learn
# !pip install sentence-transformers

In [1]:
import os

In [2]:
# Read documents from directory
from langchain_community.document_loaders import PyPDFLoader

file_path = "../docs/Personal Data English V2-23April2023- Reviewed-.pdf"
loader = PyPDFLoader(file_path)

In [3]:
docs = loader.load()
docs[0]

Ignoring wrong pointing object 6 0 (offset 0)
Ignoring wrong pointing object 36 0 (offset 0)
Ignoring wrong pointing object 57 0 (offset 0)


Document(metadata={'producer': 'macOS Version 13.1 (Build 22C65) Quartz PDFContext', 'creator': 'Word', 'creationdate': "D:20230423214602Z00'00'", 'title': 'Microsoft Word - Personal Data English V2-23April2023- Reviewed-.docx', 'moddate': "D:20230423214602Z00'00'", 'source': '../docs/Personal Data English V2-23April2023- Reviewed-.pdf', 'total_pages': 16, 'page': 0, 'page_label': '1'}, page_content='')

In [4]:
type(docs), type(docs[0])

(list, langchain_core.documents.base.Document)

In [5]:
print(len(docs))

16


In [6]:
docs[1].page_content

'Personal Data Protection Law Article 1 For the purpose of implementing this Law, the following terms shall have the meanings assigned thereto, unless the context requires otherwise: 1-Law: The Personal Data Protection Law. 2-Regulations: The Implementing Regulations of the Law. 3-Competent Authority: The authority to be determined by a resolution of the Council of Ministers. 4-Personal Data: Any data, regardless of its source or form, that may lead to identifying an individual specifically, or that may directly or indirectly make it possible to identify an individual, including name, personal identification number, addresses, contact numbers, license numbers, records, personal assets, bank and credit card numbers, photos and videos of an individual, and any other data of personal nature. 5-Processing: Any operation carried out on Personal Data by any means, whether manual or automated, including collecting, recording, saving, indexing, organizing, formatting, storing, modifying, updat

In [7]:
import pprint
pprint.pp(docs[0].metadata)

{'producer': 'macOS Version 13.1 (Build 22C65) Quartz PDFContext',
 'creator': 'Word',
 'creationdate': "D:20230423214602Z00'00'",
 'title': 'Microsoft Word - Personal Data English V2-23April2023- '
          'Reviewed-.docx',
 'moddate': "D:20230423214602Z00'00'",
 'source': '../docs/Personal Data English V2-23April2023- Reviewed-.pdf',
 'total_pages': 16,
 'page': 0,
 'page_label': '1'}


In [8]:
# Get all metadata and parameters in "Document"
pprint.pp(docs[1].__dict__.keys())

dict_keys(['id', 'metadata', 'page_content', 'type'])


In [9]:
pprint.pp(docs[1].__dict__)

{'id': None,
 'metadata': {'producer': 'macOS Version 13.1 (Build 22C65) Quartz PDFContext',
              'creator': 'Word',
              'creationdate': "D:20230423214602Z00'00'",
              'title': 'Microsoft Word - Personal Data English V2-23April2023- '
                       'Reviewed-.docx',
              'moddate': "D:20230423214602Z00'00'",
              'source': '../docs/Personal Data English V2-23April2023- '
                        'Reviewed-.pdf',
              'total_pages': 16,
              'page': 1,
              'page_label': '2'},
 'page_content': 'Personal Data Protection Law Article 1 For the purpose of '
                 'implementing this Law, the following terms shall have the '
                 'meanings assigned thereto, unless the context requires '
                 'otherwise: 1-Law: The Personal Data Protection Law. '
                 '2-Regulations: The Implementing Regulations of the Law. '
                 '3-Competent Authority: The authority to 

## Vectore Store Definition

In [10]:
from langchain_chroma.vectorstores import Chroma
from langchain_graph_retriever.transformers import ShreddingTransformer
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

# vector_store = Chroma.from_documents(
#     documents=list(ShreddingTransformer().transform_documents(animals)),
#     embedding=embeddings,
#     collection_name="animals",
# )

## Experimeting Chunking Strategy

*Page -> Article -> Paragraph*

**Metdata** 
- Regulation Name (same for all)
- Article Number
- Paragraph ID
- Version Date (same for all)
- Language (English, same for all)
- Sector: keyword mapping table + scoring — produce list of matched sectors per article (e.g. `["government","health","finance"]`) and a primary sector chosen by strongest match.

In [11]:
sectors = [
    "Government & Public Sector",
    "Health & Medical Services",
    "Finance & Banking",
    "Telecommunications & Digital Infrastructure",
    "Cybersecurity & National Security",
    "Research, Education & Statistics",
]

In [12]:
# text_splitter = RecursiveCharacterTextSplitter(
#                             chunk_size=512, chunk_overlap=64,
#                             # Seperate on occurenace of Article {number}
#                             separators=[])
# texts = text_splitter.split_text("".join([doc.page_content for doc in docs]))

In [13]:
texts = "".join([doc.page_content for doc in docs])

In [14]:
# len(texts)

In [15]:
# type(texts), type(texts[0])

In [16]:
# texts[0]
texts

"Personal Data Protection Law Article 1 For the purpose of implementing this Law, the following terms shall have the meanings assigned thereto, unless the context requires otherwise: 1-Law: The Personal Data Protection Law. 2-Regulations: The Implementing Regulations of the Law. 3-Competent Authority: The authority to be determined by a resolution of the Council of Ministers. 4-Personal Data: Any data, regardless of its source or form, that may lead to identifying an individual specifically, or that may directly or indirectly make it possible to identify an individual, including name, personal identification number, addresses, contact numbers, license numbers, records, personal assets, bank and credit card numbers, photos and videos of an individual, and any other data of personal nature. 5-Processing: Any operation carried out on Personal Data by any means, whether manual or automated, including collecting, recording, saving, indexing, organizing, formatting, storing, modifying, updat

In [27]:
# Paragraph Level Split
import re
chunks = re.split(r'(?i)(?=\bArticle\s+\d+\b)', texts)
chunks

['Personal Data Protection Law ',
 "Article 1 For the purpose of implementing this Law, the following terms shall have the meanings assigned thereto, unless the context requires otherwise: 1-Law: The Personal Data Protection Law. 2-Regulations: The Implementing Regulations of the Law. 3-Competent Authority: The authority to be determined by a resolution of the Council of Ministers. 4-Personal Data: Any data, regardless of its source or form, that may lead to identifying an individual specifically, or that may directly or indirectly make it possible to identify an individual, including name, personal identification number, addresses, contact numbers, license numbers, records, personal assets, bank and credit card numbers, photos and videos of an individual, and any other data of personal nature. 5-Processing: Any operation carried out on Personal Data by any means, whether manual or automated, including collecting, recording, saving, indexing, organizing, formatting, storing, modifying,

In [18]:
# Apply Regex to get match of every "Article {number}" in text

string = chunks[23]
import re

numbers = re.findall(r'\bArticle\s+(\d+)\b', string, flags=re.IGNORECASE)
numbers

['23']

In [19]:
# Apply Regex to get match for every "1-{number}" in text (representing paragraphs)

matches = re.findall(r'(\d)-', string)
matches

['1', '2']

In [20]:
paragraphs = re.split(r'(?=\b\d+\s*-\s*)', chunks[2])   # split where a digit(s) then hyphen start
paragraphs

['Article 2 ',
 '1-The Law applies to any Processing of Personal Data related to individuals that takes place in the Kingdom by any means, including the Processing of Personal Data related to individuals residing in the Kingdom by any means from any party outside the Kingdom. This includes the data of the deceased if it would lead to them or a member of their family being identified specifically. ',
 "2-The scope of applying the Law excludes the individual's Personal Data Processing for purposes that do not go beyond personal or family use, as long as the Data Subject did not publish or disclose it to others. The Regulations shall define personal and family use provided in this Paragraph. "]

In [21]:
re.findall(r'(\d+)-', paragraphs[0])

[]

In [ ]:
# chunks = []

In [49]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# === 1) Your sectors (human labels) ===
sectors = [
    "Government & Public Sector",
    "Health & Medical Services",
    "Finance & Banking",
    "Telecommunications & Digital Infrastructure",
    "Cybersecurity & National Security",
    "Research, Education & Statistics",
]

# === 2) Build sector profile texts (expand each sector with keywords / short phrases)
# Make these richer than just the name — repeat/phrase them to increase TF-IDF presence.
sector_keyword_map = {
    "Government & Public Sector": [
        "public entity", "competent authority", "ministry", "council of ministers",
        "public authority", "government", "regulator", "public institution"
    ],
    "Health & Medical Services": [
        "health", "medical records", "patient", "health services", "telemedicine", "clinical"
    ],
    "Finance & Banking": [
        "bank", "credit", "credit bureau", "central bank", "loan", "financial institution", "payment"
    ],
    "Telecommunications & Digital Infrastructure": [
        "telecom", "internet service provider", "isp", "network", "communications", "digital identity"
    ],
    "Cybersecurity & National Security": [
        "cybersecurity", "encryption", "security breach", "national cybersecurity authority", "incident response"
    ],
    "Research, Education & Statistics": [
        "research", "study", "statistics", "anonymized", "academic", "university", "student"
    ],
}

def get_keywords_for_sector(paragraphs: list[str]) -> list[str]:
    # Build a short 'document' for each sector by joining keywords
    sector_docs = []
    sector_labels = []
    for label in sectors:
        kws = sector_keyword_map.get(label, [])
        # repeat keywords to give them some weight relative to long paragraphs
        doc = (label + " ") + " ".join(kws) + " " + " ".join(kws)
        sector_docs.append(doc)
        sector_labels.append(label)


    # # === 3) Example paragraphs: replace this list with your actual paragraph texts ===
    # paragraphs = [
    #     # short examples (use the real extracted paragraphs)
    #     "The competent authority shall oversee processing by public entities and ensure compliance.",
    #     "Medical records and patient history must be kept confidential by the health provider.",
    #     "Banks and credit bureaus may share credit data subject to law and consent.",
    #     "Service providers and ISPs must follow rules for network data retention and user privacy.",
    #     "Controllers must implement strong encryption and report cybersecurity incidents promptly.",
    #     "Anonymized data may be used for academic research and statistics by universities."
    # ]

    # === 4) Vectorize sector docs + paragraphs together ===
    # Fit TF-IDF on the whole collection (so vocabulary includes paragraph terms)
    corpus = sector_docs + paragraphs

    vectorizer = TfidfVectorizer(
        stop_words="english",
        ngram_range=(1,2),  # unigrams + bigrams often useful for domain terms
        max_df=0.95,
        min_df=1
    )
    tfidf_matrix = vectorizer.fit_transform(corpus)  # shape (n_sectors + n_paragraphs, n_features)

    n_sectors = len(sector_docs)
    sector_vecs = tfidf_matrix[:n_sectors]       # sector profile vectors
    para_vecs   = tfidf_matrix[n_sectors:]       # paragraph vectors

    # === 5) Compute cosine similarity between each paragraph and each sector profile ===
    # result shape: (n_paragraphs, n_sectors)
    sims = cosine_similarity(para_vecs, sector_vecs)
    return sims, sector_labels, n_sectors

# === 6) Multi-label assignment by threshold (and optionally top_k) ===
def assign_labels_for_paragraph(sim_row, labels, threshold=0.05, top_k=None):
    """
    sim_row: 1D array of cosine similarities for one paragraph (len == n_sectors)
    labels: list of sector labels in the same order as sim_row
    threshold: minimum cosine similarity to consider a label
    top_k: if set, always pick top_k labels (if their scores meet threshold)
    """
    # find indices meeting threshold
    idxs = np.where(sim_row >= threshold)[0].tolist()
    # From sim_row, select up to 2 indixes with max similarity scores
    # idxs = []
    # sorted_idxs = np.argsort(-sim_row)  # descending
    # # Select highest two (ignore threshold for now)
    # for idx in sorted_idxs[:2]:
    #     if sim_row[idx] >= threshold:
    #         idxs.append(idx)
    

    # optionally add top_k if none meet threshold or to ensure we get top_k (but only if above a lower min)
    # if top_k:
    #     sorted_idxs = np.argsort(-sim_row)  # descending
    #     chosen = []
    #     # enforce a minimum soft threshold (e.g., 0.05) to avoid picking irrelevant tags when sim is near zero
    #     soft_min = 0.05
    #     for idx in sorted_idxs[:top_k]:
    #         if sim_row[idx] >= 0:
    #             chosen.append(idx)
    #     idxs = sorted(set(idxs) | set(chosen))
    
    # build result list of (label, score) sorted by score descending
    result = sorted([(labels[i], float(sim_row[i])) for i in idxs], key=lambda x: -x[1])
    return result

# Example: assign labels for all paragraphs
threshold = 0.01   # starting recommendation; tune later (see notes)
top_k = None       # or set to 1/2 if you always want at least top-1
assigned = []
sims, sector_labels, n_sectors = get_keywords_for_sector(chunks)
print(sims.shape)
# print(sector_labels)
# print(n_sectors)
for i, sim_row in enumerate(sims):
    # print(sim_row)
    labels_and_scores = assign_labels_for_paragraph(sim_row, sector_labels, threshold=threshold, top_k=top_k)
    assigned.append({
        "paragraph": chunks[i],
        "labels": labels_and_scores,
        "similarities": {sector_labels[j]: float(sim_row[j]) for j in range(n_sectors)}
    })

# Print results
for a in assigned:
    print("PARA:", a["paragraph"])
    print("ASSIGNED:", a["labels"])
    print("----")


(44, 6)
PARA: Personal Data Protection Law 
ASSIGNED: []
----
PARA: Article 1 For the purpose of implementing this Law, the following terms shall have the meanings assigned thereto, unless the context requires otherwise: 1-Law: The Personal Data Protection Law. 2-Regulations: The Implementing Regulations of the Law. 3-Competent Authority: The authority to be determined by a resolution of the Council of Ministers. 4-Personal Data: Any data, regardless of its source or form, that may lead to identifying an individual specifically, or that may directly or indirectly make it possible to identify an individual, including name, personal identification number, addresses, contact numbers, license numbers, records, personal assets, bank and credit card numbers, photos and videos of an individual, and any other data of personal nature. 5-Processing: Any operation carried out on Personal Data by any means, whether manual or automated, including collecting, recording, saving, indexing, organizing,

In [51]:
def get_sector(index) -> list[str]:
    sectors = assigned[index]["labels"]
    return [label for label, score in sectors]

In [55]:
from langchain_core.documents import Document

def chunk_by_paragraph(docs: list[str]) -> list[Document]:
    
    regulation_name = docs[0].strip()
    version_date = "27/03/2023"
    language = "English"
    curr_article = 1
    # current_paragraph = 1
    chunked_docs = []

    while curr_article < len(docs):
        
        page = docs[curr_article]
        paragraphs = re.split(r'(?=\b\d+\s*-\s*)', page)   # split where a digit(s) then hyphen start

        for para in paragraphs:

            # Extract paragraph number
            current_paragraph = re.findall(r'(\d+)-', para)
            current_paragraph = current_paragraph[0] if current_paragraph else ""

            if para.strip():  # Avoid empty paragraphs
                chunked_docs.append(
                    Document(
                        page_content=para.strip(),
                        metadata={"regulation name": regulation_name,
                                  "article number": curr_article,
                                  "paragraph number": current_paragraph,
                                  "version datae": version_date,
                                  "language": language,
                                  "sector": get_sector(curr_article)
                                  }
                    )
                )
        curr_article += 1
    
    return chunked_docs

In [ ]:
chunked_docs = chunk_by_paragraph(chunks)

In [61]:
chunked_docs[5].metadata

{'regulation name': 'Personal Data Protection Law',
 'article number': 1,
 'paragraph number': '5',
 'version datae': '27/03/2023',
 'language': 'English',
 'sector': ['Government & Public Sector',
  'Health & Medical Services',
  'Finance & Banking']}

In [ ]:
vector_store = Chroma.from_documents(
    documents=list(ShreddingTransformer().transform_documents(chunked_docs)),
    embedding=HuggingFaceEmbeddings(),
    collection_name="Personal_Data_Protection_Law_en",
    persist_directory="../chroma_langchain_db",

)

/var/folders/l7/m3wgz92x6vj1gkr9lwh0893c0000gn/T/ipykernel_3510/2282824174.py:3: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embedding=HuggingFaceEmbeddings(),


RuntimeError: Numpy is not available